<a href="https://colab.research.google.com/github/sukstar76/kaggle_titanic/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')


In [0]:
import pandas as pd

In [0]:
train = pd.read_csv('/gdrive/My Drive/titanic/train.csv')
test = pd.read_csv('/gdrive/My Drive/titanic/test.csv')
total_set = [train,test]

In [0]:
for dataset in total_set:
  dataset['Sex'] = dataset['Sex'].map({'female':1,'male':0}).astype(int)

  dataset['Name']=dataset['Name'].map(lambda x: x.split(',')[1].split('.')[0].strip())
  dataset['Name'] = dataset['Name'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
  dataset['Name'] = dataset['Name'].replace('Mlle', 'Miss')
  dataset['Name'] = dataset['Name'].replace('Ms', 'Miss')
  dataset['Name'] = dataset['Name'].replace('Mme', 'Mrs')
  dataset['Name'] = dataset['Name'].map({'Mr':1,'Miss':2,'Mrs':3,'Master':4,'Other':5})
  dataset['Name'] = dataset['Name'].fillna(0)
  
  dataset['Embarked'] = dataset['Embarked'].fillna('S')
  dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

  dataset['Age'].fillna(dataset['Age'].mean(), inplace=True)
  dataset['Age'] = dataset['Age'].astype(int)
  dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
  dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
  dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
  dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
  dataset.loc[ dataset['Age'] > 64, 'Age'] = 4

  dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
  dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
  dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
  dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
  dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
  dataset['Fare'] = dataset['Fare'].astype(int)

  dataset["Family"] = dataset["Parch"] + dataset["SibSp"]
  dataset['Family'] = dataset['Family'].astype(int)
  dataset.loc[dataset['Family'] == 0, 'Family'] = 0
  dataset.loc[dataset['Family'] >= 1, 'Family'] = 1

In [0]:
y =train['Survived']
train = train.drop(['SibSp','Parch','Ticket','Cabin'],axis =1)
test = test.drop(['SibSp','Parch','Ticket','Cabin'],axis =1)

x_train = train.drop(['PassengerId','Survived'],axis=1)
x_test = test.drop(['PassengerId'],axis=1).copy()


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier ,GradientBoostingClassifier
from sklearn.model_selection import KFold,GridSearchCV
##log_clf = LogisticRegression()
kfold = KFold(n_splits=10)

svc_clf = SVC(probability=True)
svc_param = {'kernel':['rbf'], 'gamma':[0.001,0.01,0.1,1],'C':[1,50,100,200,500,1000] }
gs_svc = GridSearchCV(svc_clf,param_grid = svc_param, cv=kfold, scoring="accuracy", verbose = 1)
gs_svc.fit(x_train,y)
svc_best = gs_svc.best_estimator_
##dt_clf = DecisionTreeClassifier()
gb_clf = GradientBoostingClassifier()
gb_param = {'loss' : ["deviance"],'n_estimators' : [100,200,300],'learning_rate': [0.1, 0.05, 0.01],'max_depth': [4, 8],'min_samples_leaf': [100,150]}
gs_gbc = GridSearchCV(gb_clf,param_grid = gb_param, cv=kfold, scoring="accuracy",verbose = 1)
gs_gbc.fit(x_train,y)
gs_best = gs_gbc.best_estimator_

rf_clf = RandomForestClassifier(n_estimators=100)
rf_param = {"max_depth": [None],"min_samples_split": [2, 3,10],"min_samples_leaf": [1, 3, 10],"bootstrap": [False],"n_estimators" :[100,300],"criterion": ["gini"]}
gs_rf = GridSearchCV(rf_clf,param_grid = rf_param, cv=kfold, scoring="accuracy", verbose = 1)
gs_rf.fit(x_train,y)
rf_best = gs_rf.best_estimator_

voting_clf = VotingClassifier(estimators=[('svc',svc_best),('rf',rf_best),('gs',gs_best)],voting='soft')
voting_clf.fit(x_train,y)
print(voting_clf.score(x_train,y))


In [0]:
 predict = voting_clf.predict(x_test)
 print(predict.shape)

In [0]:
submission = pd.DataFrame(test["PassengerId"],columns=['PassengerId'])
submission['Survived']=predict

submission.head()

submission.to_csv('/gdrive/My Drive/titanic/submisiion.csv', index=False)